In [89]:
import os, sys
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFE

from numpy import set_printoptions
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB



In [90]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/feature1"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

* kiểm tra và tạo các thư mục (nếu chưa có)
* tập test 33%

# 5. Xây dựng và đánh giá mô hình học sâu Neutual Network

In [91]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{exps_dir}/feature1/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 1.0, 1: 1.0}

* Sử dụng thư viện **imbalanced-learn** để thực hiện Oversampling bằng phương pháp **SMOTE (Synthetic Minority Over-sampling Technique).** 

#### Giải thích:
* Vì tập dữ liệu của nhóm là tập dữ liệu phân loại không cân bằng => **SMOTE** được sử dụng để tăng cường dữ liệu trong trường hợp mẫu của lớp thiểu số (minority class) quá ít so với lớp đa số (majority class), giúp cân bằng dữ liệu và cải thiện hiệu suất của mô hình.

In [92]:
x_test.shape

(330, 39)

In [93]:
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# model = Sequential()
# model.add(Dense(512,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# # Thêm các lớp ẩn
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(32, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# # Lớp đầu ra với activation function 'sigmoid' cho bài toán phân lớp nhị phân
# model.add(Dense(1, activation='tanh'))

# # Biên dịch mô hình
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Giải thích:
   Nhóm sử dụng thư viện TensorFlow và Keras xây dựng một mô hình mạng nơ-ron sử dụng Sequential API:
   
   1. Đầu tiên, tạo mô hình Sử dụng mô hình tuần tự **Sequential**
   2. Sau đó, thêm các lớp Dense với hàm kích hoạt **'relu'**. Nhóm sử dụng **Dropout** tại mỗi tầng để ngẫu nhiên "tắt" khoảng 50% số nơ-ron tại tầng đó trong quá trình huấn luyện, ngăn chặn hiện tượng quá mức học (overfitting).
   3. Thêm lớp output với hàm kích hoạt 'tanh'. Hàm **'tanh'** chuyển đổi mỗi giá trị đầu ra từ khoảng (-∞, ∞) về khoảng (-1, 1).
   4. Compile mô hình: 
      * loss=**binary_crossentropy** được sử dụng vì đây là một bài toán phân loại nhị phân. 
      * optimizer=**adam** là một thuật toán tối ưu hóa phổ biến. 
      * metric=**'accuracy'**, Theo dõi độ chính xác của mô hình trong quá trình huấn luyện.

In [94]:

earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=0.00001,
    verbose=1,
)

callbacks = [early_stopping,lr_scheduler]

checkpointer = ModelCheckpoint(filepath = "Emotion_weights.hdf5", verbose = 1, save_best_only=True)





In [100]:
from keras.models import Sequential
from keras.layers import Dense, Concatenate
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Tạo dữ liệu mẫu
# Giả sử x_train, y_train là dữ liệu huấn luyện ban đầu

# Sử dụng SMOTE để oversample dữ liệu

# Tạo các mô hình cơ sở
rf_model = RandomForestClassifier()
gb_model = GradientBoostingClassifier()

# Huấn luyện các mô hình cơ sở
rf_model.fit(x_train, y_train)
gb_model.fit(x_train, y_train)

# Dự đoán trên tập kiểm thử
rf_pred = rf_model.predict(x_test)
gb_pred = gb_model.predict(x_test)

# Kết hợp đầu ra của các mô hình cơ sở
stacked_input = np.column_stack((rf_pred, gb_pred))

# Xây dựng mô hình học sâu để học từ đầu đầu ra của các mô hình cơ sở


stacked_model = Sequential()
stacked_model.add(Dense(512, activation='relu',input_dim=stacked_input.shape[1]))
stacked_model.add(BatchNormalization())
stacked_model.add(Dropout(0.5))

# Thêm các lớp ẩn
stacked_model.add(Dense(256, activation='relu'))
stacked_model.add(BatchNormalization())
stacked_model.add(Dropout(0.5))

stacked_model.add(Dense(128, activation='relu'))
stacked_model.add(BatchNormalization())
stacked_model.add(Dropout(0.5))

stacked_model.add(Dense(64, activation='relu'))
stacked_model.add(BatchNormalization())
stacked_model.add(Dropout(0.5))

stacked_model.add(Dense(32, activation='relu'))
stacked_model.add(BatchNormalization())
stacked_model.add(Dropout(0.5))

# Lớp đầu ra với activation function 'sigmoid' cho bài toán phân lớp nhị phân
stacked_model.add(Dense(1, activation='sigmoid'))

stacked_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình stacked trên dữ liệu đầu ra của các mô hình cơ sở
stacked_model.fit(stacked_input, y_test, epochs=50, batch_size=32, validation_split=0.2)

# Dự đoán và đánh giá mô hình stacked
stacked_pred = stacked_model.predict(stacked_input)
stacked_pred_binary = (stacked_pred > 0.5).astype(int)
accuracy_stacked = accuracy_score(y_test, stacked_pred_binary)
print(f'Accuracy of Stacked Model: {accuracy_stacked}')


Epoch 1/50
9/9 [==============================] - 1s 33ms/step - loss: 1.0053 - accuracy: 0.5076 - val_loss: 0.6717 - val_accuracy: 0.7273
Epoch 2/50
9/9 [==============================] - 0s 7ms/step - loss: 0.9533 - accuracy: 0.5682 - val_loss: 0.6469 - val_accuracy: 0.7273
Epoch 3/50
9/9 [==============================] - 0s 5ms/step - loss: 0.8436 - accuracy: 0.6402 - val_loss: 0.6264 - val_accuracy: 0.7273
Epoch 4/50
9/9 [==============================] - 0s 7ms/step - loss: 0.8697 - accuracy: 0.6250 - val_loss: 0.6122 - val_accuracy: 0.7273
Epoch 5/50
9/9 [==============================] - 0s 7ms/step - loss: 0.8302 - accuracy: 0.6439 - val_loss: 0.6010 - val_accuracy: 0.7273
Epoch 6/50
9/9 [==============================] - 0s 5ms/step - loss: 0.7850 - accuracy: 0.6477 - val_loss: 0.5933 - val_accuracy: 0.7273
Epoch 7/50
9/9 [==============================] - 0s 5ms/step - loss: 0.7552 - accuracy: 0.6667 - val_loss: 0.5865 - val_accuracy: 0.7273
Epoch 8/50
9/9 [=================

In [101]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
rf_test_pred = rf_model.predict(x_test)
gb_test_pred = gb_model.predict(x_test)

# Kết hợp đầu ra của các mô hình cơ sở
stacked_test_input = np.column_stack((rf_test_pred, gb_test_pred))
stacked_test_pred = stacked_model.predict(stacked_test_input)
stacked_test_pred_binary = (stacked_test_pred > 0.5).astype(int)

# Đánh giá cho mô hình cơ sở - Random Forest
accuracy_rf = accuracy_score(y_test, rf_test_pred)
print(f'Accuracy of Random Forest Model: {accuracy_rf}')
print("Classification Report for Random Forest:")
print(classification_report(y_test, rf_test_pred))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test, rf_test_pred))

# Đánh giá cho mô hình cơ sở - Gradient Boosting
accuracy_gb = accuracy_score(y_test, gb_test_pred)
print(f'Accuracy of Gradient Boosting Model: {accuracy_gb}')
print("Classification Report for Gradient Boosting:")
print(classification_report(y_test, gb_test_pred))
print("Confusion Matrix for Gradient Boosting:")
print(confusion_matrix(y_test, gb_test_pred))

# Đánh giá cho mô hình stacked
accuracy_stacked = accuracy_score(y_test, stacked_test_pred_binary)
print(f'Accuracy of Stacked Model: {accuracy_stacked}')
print("Classification Report for Stacked Model:")
print(classification_report(y_test, stacked_test_pred_binary))
print("Confusion Matrix for Stacked Model:")
print(confusion_matrix(y_test, stacked_test_pred_binary))

11/11 [==============================] - 0s 1ms/step
Accuracy of Random Forest Model: 0.7818181818181819
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       242
           1       0.59      0.60      0.60        88

    accuracy                           0.78       330
   macro avg       0.72      0.72      0.72       330
weighted avg       0.78      0.78      0.78       330

Confusion Matrix for Random Forest:
[[205  37]
 [ 35  53]]
Accuracy of Gradient Boosting Model: 0.8
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.92      0.80      0.85       242
           1       0.59      0.80      0.68        88

    accuracy                           0.80       330
   macro avg       0.75      0.80      0.77       330
weighted avg       0.83      0.80      0.81       330

Confusion Matrix for Gradient Boosting:
[[194  48]
 [ 1

### Note:
   #### 1. Early Stopping:
   * **monitor='val_accuracy'**: Đây là đại lượng mà quá trình Early Stopping sẽ theo dõi. Trong trường hợp này, nó sẽ kiểm tra độ chính xác trên tập validation.
   * **min_delta=0.00005**: Số lượng thay đổi nhỏ nhất giữa hai epoch để coi là có sự cải thiện. Nếu không có sự cải thiện lớn hơn min_delta sau số lượng epoch 'patience', quá trình huấn luyện sẽ dừng lại.
   * **patience=11**: Số epoch mà mô hình có thể không cải thiện trước khi quá trình Early Stopping được kích hoạt.
   * **verbose=1**: Hiển thị thông báo khi quá trình Early Stopping được kích hoạt.
   * **restore_best_weights=True**: Khôi phục trọng số của mô hình tại epoch có độ chính xác cao nhất trên tập validation.
    
   #### 2. Learning Rate Scheduler (ReduceLROnPlateau):
   * **monitor='val_accuracy'**: Đại lượng mà Learning Rate Scheduler sẽ theo dõi, cũng là độ chính xác trên tập validation.
   * **factor=0.5**: Hệ số giảm learning rate. Khi được kích hoạt (sau 'patience' epochs mà không có cải thiện), learning rate sẽ được giảm đi một lượng bằng factor.
   * **patience=7**: Số epoch mà mô hình có thể không cải thiện trước khi learning rate được giảm.
   * **min_lr=0.00001**: Giới hạn dưới cho learning rate sau khi được giảm. Không bao giờ giảm dưới giá trị này.
   * **verbose=1**: Hiển thị thông báo khi quá trình giảm learning rate được kích hoạt.

### Giải thích:
   Nhóm thực hiện quá trình huấn luyện mô hình máy học với **model.fit**:
   
   * **x=x_train, y=y_train:** Dữ liệu đầu vào và nhãn cho quá trình đào tạo.
   * **validation_data=(x_test,y_test):** Dữ liệu kiểm tra.
   * **class_weight=class_weights_dict:** Trọng số lớp được áp dụng cho cân bằng lớp. (-> Vì tập dữ liệu mất cân bằng lớp)
   * Kích thước batch là **batch_size=128.** 
   * Số lượng epochs là **epochs=600.** 
   * Nhóm sử dụng **callbacks = [early_stopping,lr_scheduler]** để đảm bảo rằng mô hình được lưu lại tại những điểm có độ chính xác tốt nhất trên tập kiểm tra và đồng thời ngừng sớm nếu không có sự cải thiện đáng kể. Bên cạnh đó, giảm learning rate giúp mô hình hội tụ chậm hơn và tránh việc vượt qua điểm tối ưu toàn cục.

#### Biểu đồ thể hiện loss và accuracy

   * Hàm loss biểu thị mức độ lỗi của mô hình trong quá trình học tập
   * Hàm accuracy biểu thị mức độ chính xác của mô hình trong việc dự đoán các dữ liệu trong tập kiểm tra.

In [98]:
probabilities = stacked_model.predict(x_test)
predictions = np.argmax(probabilities, axis=-1)
stacked_model.evaluate(x_test,y_test)

ValueError: in user code:

    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\PC\miniconda3\envs\test\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibilit

In [ ]:
model.evaluate(x_test,y_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

AttributeError: 'GradientBoostingClassifier' object has no attribute 'evaluate'

### Nhận xét

   * Ta thấy **accuracy: 0.74**. Đây là tỷ lệ mẫu được dự đoán đúng trên tổng số mẫu. Tuy nhiên, vì tập dữ liệu của nhóm mất cân bằng lớp (lớp 0 có 239 mẫu, trong khi lớp 1 (gian lận) chỉ có 85 mẫu), Accuracy có thể không phản ánh chính xác vì mô hình có thể dự đoán một lớp nhiều hơn.
   
   * Các chỉ số **precision, recall, f1-score của lớp 1 đều =0.00**, Điều này có nghĩa là mô hình không tìm thấy dự đoán đúng cho lớp gian lận, không đạt được hiệu suất tốt trên lớp ít xuất hiện này. => Mô hình có vấn đề trong việc dự đoán lớp 1. 

### Kết luận:
- Nhóm xây dựng một mạng Netual network ứng dụng kỹ thuật trong học sâu như callbacks, đánh trọng số,.... Nhưng không khắc phục được tình trạng (imbalanced variable).

In [ ]:
def check_result(prediction):
    if isinstance(prediction, (np.ndarray, pd.Series)):
        prediction = prediction.item()  # Hoặc sử dụng prediction[0] hoặc prediction.iloc[0]
    if prediction==1:
        return "Y"
    else:
        return "N"

In [ ]:
def check_output(stt,model, X_test, Y_test):
    print(f"Lần thứ {stt}: ")
    indx = rd.randint(0, X_test.shape[0] - 1)  # Đảm bảo indx nằm trong giới hạn của DataFrame
    sample = X_test.iloc[indx, :].values        # Chuyển DataFrame thành mảng NumPy
    sample = np.expand_dims(sample, axis=0)     # Mở rộng kích thước cho phù hợp với mô hình
    y_predict = model.predict(sample)
    print(y_predict)
    Y_check = check_result(Y_test.iloc[indx])
    print('True:', Y_check)
    print('Predicted:', check_result(y_predict))
    print("----------------------------")
    if Y_check==check_result(y_predict):
        return True
    else: return False
    
    

In [ ]:
solandung=0
hopdonggianlan=0
for indx in range(len(x_test)):
    sample = x_test.iloc[indx, :].values        # Chuyển DataFrame thành mảng NumPy
    sample = np.expand_dims(sample, axis=0)     # Mở rộng kích thước cho phù hợp với mô hình
    y_predict = model.predict(sample)
    #print(y_predict[0][0])
    Y_check = check_result(y_test.iloc[indx])
    print('True:', Y_check)
    print('Predicted:', check_result(y_predict))
    print("----------------------------")
    if Y_check==check_result(y_predict):
        solandung+=1
        if Y_check=='Y':
            hopdonggianlan+=1
    

True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: Y
----------------------------
True: Y
Predicted: Y
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: Y
Predicted: Y
----------------------------
True: Y
Predicted: Y
----------------------------
True: N
Predicted: N
----------------------------
True: N
Predicted: N
----------------------------
True: Y
Predicted: N
----------------------------
True: Y
Predicted: Y
----------------------------
True: Y
Predicted: N
----------------------------


In [ ]:
print(f"Số lần dự đoán đúng: {solandung} chiếm khoảng {(solandung/len(x_test))*100}% tổng lần dự đoán")
print(f"Số hợp đồng gian lận trong lần kiểm tra: {hopdonggianlan} chiếm khoảng {(hopdonggianlan/len(y_test[y_test['fraud_reported']==1]))*100}% so với thực tế")


Số lần dự đoán đúng: 264 chiếm khoảng 80.0% tổng lần dự đoán
Số hợp đồng gian lận trong lần kiểm tra: 70 chiếm khoảng 79.54545454545455% so với thực tế


* Ta có thể thấy trong tất cả các lần dự đoán, mô hình đều dự đoán 'N'(lớp 0). Đó là lí do tại sao chỉ số 'accuracy'lại cao như vậy ( vốn tập dữ liệu bị mất cân bằng lớp lớp 0 gấp 3 lần lớp 1)

#### => Kết luận: Ta không dựa vào chỉ số 'accuracy' để đánh giá các mô hình.